<font color="#CA0032"><h1 align="left">**Redes Generativas Adversariales Condicionales (C-GANs)**</h1></font>

<font color="#6E6E6E"><h1 align="left">**Creación de caracteres manuscritos nuevos entrenando con MNIST:**</h1></font>

<font color="#6E6E6E"><h1 align="left">**Modelos Deep Learning**</h1></font>

<h2 align="left">Manuel Sánchez-Montañés</h2>

<font color="#6E6E6E"><h2 align="left">manuel.smontanes@gmail.com</h2></font>

In [ ]:
# Basado en:
#
# https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/

**Para más información:**

**Libros:**

- Capítulo 20 ("Deep Generative Models") de "Deep Learning" (2016): https://amzn.to/2YuwVjL

- Capítulo 8 ("Generative Deep Learning") de "Deep Learning with Python" (2017): https://amzn.to/2U2bHuP

**Artículos:**

- Generative Adversarial Networks (2014): https://arxiv.org/abs/1406.2661

- Tutorial: Generative Adversarial Networks, NIPS (2016): https://arxiv.org/abs/1701.00160

- Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks (2015): https://arxiv.org/abs/1511.06434

- Conditional Generative Adversarial Nets (2014): https://arxiv.org/abs/1411.1784

- Image-To-Image Translation With Conditional Adversarial Networks (2017): https://arxiv.org/abs/1611.07004

- Conditional Generative Adversarial Nets For Convolutional Face Generation (2015): https://www.foldl.me/uploads/2015/conditional-gans-face-generation/paper.pdf

In [ ]:
COLAB    = True
GRAPHVIZ = True

In [ ]:
# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU
from keras.layers import Dropout, Embedding, Concatenate

import matplotlib.pyplot as plt

In [ ]:
# load fashion mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return [X, trainy]

In [ ]:
# load image data
dataset = load_real_samples()
X, y = dataset

In [ ]:
X.shape, y.shape

In [ ]:
n_ejemplos = 15

for i in randint(low=0, high=len(X), size=n_ejemplos):
    plt.imshow(X[i].reshape(28,28)) # cmap="gray"
    plt.title("Clase: {}".format(y[i]))
    plt.show()

In [ ]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((7, 7, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 32 * 7 * 7
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((7, 7, 32))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 14x14
	gen = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same')(merge)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 28x28
	gen = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(32, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(32, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

In [ ]:
# size of the latent space
latent_dim = 100

# create the generator
generator = define_generator(latent_dim)
generator.summary()

In [ ]:
if GRAPHVIZ:
    from IPython.display import SVG,display
    from keras.utils.vis_utils import model_to_dot
    if COLAB:
        display(SVG(model_to_dot(generator,show_shapes=True,dpi=64).create(prog='dot', format='svg')))
    else:
        display(SVG(model_to_dot(generator,show_shapes=True).create(prog='dot', format='svg')))

In [ ]:
# create the discriminator
discriminator = define_discriminator()

discriminator.summary()

In [ ]:
if GRAPHVIZ:
    from IPython.display import SVG,display
    from keras.utils.vis_utils import model_to_dot
    if COLAB:
        display(SVG(model_to_dot(discriminator,show_shapes=True,dpi=64).create(prog='dot', format='svg')))
    else:
        display(SVG(model_to_dot(discriminator,show_shapes=True).create(prog='dot', format='svg')))

In [ ]:
# create the gan
gan = define_gan(generator, discriminator)

gan.summary()

In [ ]:
if GRAPHVIZ:
    from IPython.display import SVG,display
    from keras.utils.vis_utils import model_to_dot
    if COLAB:
        display(SVG(model_to_dot(gan,show_shapes=True,dpi=64).create(prog='dot', format='svg')))
    else:
        display(SVG(model_to_dot(gan,show_shapes=True).create(prog='dot', format='svg')))

In [ ]:
# example of loading the generator model and generating images
from keras.models import load_model

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# create and save a plot of generated images
def show_plot(examples, n=10):
	# plot images
	for i in range(n * n):
		# define subplot
		plt.subplot(n, n, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i, :, :, 0], cmap='gray_r')
	plt.show()

def plotGeneratedImages(generator, latent_dim, order_class=False):
		latent_points, labels = generate_latent_points(latent_dim, 100)
		if order_class:
				labels = asarray([x for _ in range(10) for x in range(10)])
		# generate images
		X  = generator.predict([latent_points, labels])
		# scale from [-1,1] to [0,1]
		X = (X + 1) / 2.0
		# plot the result
		show_plot(X)

In [ ]:
# train the generator and discriminator
n_epochs=100
n_batch=128

bat_per_epo = int(dataset[0].shape[0] / n_batch)
half_batch = int(n_batch / 2)
# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    print("EPOCH", i+1)
    for j in range(bat_per_epo):
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        d_loss1, _ = discriminator.train_on_batch([X_real, labels_real], y_real)
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(generator, latent_dim, half_batch)
        # update discriminator model weights
        d_loss2, _ = discriminator.train_on_batch([X_fake, labels], y_fake)
        # prepare points in latent space as input for the generator
        [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss = gan.train_on_batch([z_input, labels_input], y_gan)
        # summarize loss on this batch
        if i%5 == 0:
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
              (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    if i%5 == 0:
        plotGeneratedImages(generator, latent_dim)

In [ ]:
# save the models

generator.save("mnist_cgan_generator.h5")
generator.save_weights("mnist_cgan_generator_weights.h5")
discriminator.save("mnist_cgan_discriminator.h5")
discriminator.save_weights("mnist_cgan_discriminator_weights.h5")
gan.save("mnist_cgan_gan.h5")
gan.save_weights("mnist_cgan_gan_weights.h5")

In [ ]:
!ls -la

### **Generación de imágenes nuevas**

In [ ]:
# load model
model = load_model('mnist_cgan_generator.h5')

In [ ]:
plotGeneratedImages(model, latent_dim, order_class=True)

In [ ]:
if COLAB:
    from google.colab import files
    files.download('mnist_cgan_generator.h5')
    files.download('mnist_cgan_generator_weights.h5')
    files.download('mnist_cgan_discriminator.h5')
    files.download('mnist_cgan_discriminator_weights.h5')
    files.download('mnist_cgan_gan.h5')
    files.download('mnist_cgan_gan_weights.h5')

Ejercicios:

- Adaptar el ejercicio a la base de datos FMNIST
- Adaptar el ejercicio a la base de datos CIFAR-10

In [ ]:
from keras.datasets.fashion_mnist import load_data

In [ ]:
from tensorflow.keras.datasets.cifar10 import load_data as load_data_cifar10

In [ ]:
?load_data_cifar10